# LSSTCam pointing errors
## Looking at offsets more days.

Craig Lage - 18-Aug-25

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord, angular_separation
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors

In [ ]:
butler = Butler('/repo/embargo', collections=["LSSTCam/raw/all", "LSSTCam/calib", \
                                        "LSSTCam/runs/nightlyValidation"])
instrument = 'LSSTCam'
camera = LsstCam.getCamera()
client = makeEfdClient()

## Get the data

In [ ]:
startDay = 20251026
endDay = 20251026

detector = camera['R22_S11']
bbox = detector.getBBox()

els = []
azs = []
point_ras = []
point_decs = []
true_ras = []
true_decs = []
deltaRas = []
deltaDecs = []
rots = []

filters = []
pressures = []
temps = []
hums = []
times = []
dayObs = startDay
expIds = []
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science']:
            continue
        try:
            calExp = butler.get('preliminary_visit_image', detector=94, visit=record.id, instrument=instrument)
            cWcs = calExp.getWcs()
            if cWcs == None:
                print(f"{record.id} had no cWcs.")
                continue
            (mountErrors, mountData) = calculateMountErrors(record, client)
            efd_azs = mountData.azimuthData['actualPosition'].values
            point_az = (efd_azs[0] + efd_azs[-1]) / 2.0
            efd_els = mountData.elevationData['actualPosition'].values
            point_el = (efd_els[0] + efd_els[-1]) / 2.0
            efd_rot = mountData.rotationData['actualPosition'].values
            rot = (efd_rot[0] + efd_rot[-1]) / 2.0
            rots.append(rot)
            rawExp = butler.get('raw', detector=94, exposure=record.id, instrument=instrument)
            md = rawExp.getMetadata()
            calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
            true_ra = calExpSkyCenter.getRa().asDegrees()
            true_dec = calExpSkyCenter.getDec().asDegrees()
            expIds.append(record.id)
            els.append(point_el)
            azs.append(point_az)
            filters.append(md['FILTBAND'])
            pressures.append(md['PRESSURE'])
            temps.append(md['AIRTEMP'])
            hums.append(md['HUMIDITY'])
            times.append((md['MJD-BEG'] + md['MJD-END']) / 2.0)
            true_ras.append(true_ra)
            true_decs.append(true_dec)
            point_ra = md['RASTART']
            point_dec = md['DECSTART']
            point_ras.append(point_ra)
            point_decs.append(point_dec)
            deltaRa = (true_ra - point_ra) * 3600.0
            deltaDec = (true_dec - point_dec) * 3600.0
            deltaRas.append(deltaRa)
            deltaDecs.append(deltaDec)
            print(record.id, deltaRa, deltaDec)
        except:
            print(f"{record.id} failed!")
            continue
    print(dayObs, len(true_ras))
    dayObs = calcNextDay(dayObs)

filename = f"/home/c/cslage/u/MTMount/mount_plots/pointing_results_rot_{startDay}_{endDay}.pkl"
with open(filename, 'wb') as f:
    pkl.dump([expIds, els, azs, rots, true_ras, true_decs, point_ras, point_decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters], f)


In [ ]:
startDay = 20250715
endDay = 20250805

filename = f"/home/c/cslage/u/MTMount/mount_plots/pointing_results_rot_{startDay}_{endDay}.pkl"
with open(filename, 'rb') as f:
    [expIds, els, azs, rots, true_ras, true_decs, point_ras, point_decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters] = pkl.load(f)

startDay2 = 20250806
endDay2 = 20250817

filename = f"/home/c/cslage/u/MTMount/mount_plots/pointing_results_rot_{startDay2}_{endDay2}.pkl"
with open(filename, 'rb') as f:
    [expIds2, els2, azs2, rots2, true_ras2, true_decs2, point_ras2, point_decs2, deltaRas2, deltaDecs2, pressures2, temps2, hums2, times2, filters2] = pkl.load(f)
expIds += expIds2; els += els2; azs += azs2; rots += rots2; true_ras += true_ras2; true_decs += true_decs2
point_ras += point_ras2; point_decs += point_decs2; deltaRas += deltaRas2; deltaDecs += deltaDecs2
pressures += pressures2; temps += temps2; hums += hums2; times += times2; filters += filters2
len(els)

In [ ]:
startDay = 20250827
endDay = 20250827

filename = f"/home/c/cslage/u/MTMount/mount_plots/pointing_results_rot_{startDay}_{endDay}.pkl"
with open(filename, 'rb') as f:
    [expIds, els, azs, rots, true_ras, true_decs, point_ras, point_decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters] = pkl.load(f)

len(els)

In [ ]:
dayObs = startDay
separation = []
for i in range(len(point_ras)):
    sep = angular_separation(point_ras[i] * u.deg, point_decs[i] * u.deg, 
                             true_ras[i] * u.deg, true_decs[i] * u.deg)
    # Returns in radians
    separation.append(sep.value * 180.0 / np.pi * 3600.0)
        
plt.hist(separation, bins = 50, range=(0,200), alpha=0.5)

plt.xlim(0, 200)
plt.legend()
plt.title(f"On sky pointing model error, {dayObs}")
plt.xlabel("Error (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/OnSky_Pointing_Errors_{dayObs}.png")

In [ ]:
sep.value * 180.0 / np.pi * 3600.0

In [ ]:
seqNums = np.array(expIds) - startDay * 1E5
plt.scatter(seqNums, separation)
plt.title(f"On sky pointing model error, {dayObs}")
plt.ylabel("Error (arcseconds)")
plt.xlabel("Sequence number")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/OnSky_Pointing_Error_Trend_{dayObs}.png")

In [ ]:
startDay = 20250715
endDay = 20250805
dayObs = startDay

while dayObs <= endDay:
    separation = []
    for i in range(len(point_ras)):
        if (expIds[i] > dayObs * 1E5) and (expIds[i] < calcNextDay(dayObs) * 1E5):
            sep = np.sqrt(((deltaRas[i] * np.cos(point_decs[i] * np.pi / 180.0))**2 + deltaDecs[i]**2))
            separation.append(sep)

    print(dayObs, len(separation), np.median(separation))
    dayObs = calcNextDay(dayObs)


In [ ]:
dayObs = startDay

while dayObs <= endDay:
    separation = []
    for i in range(len(point_ras)):
        if int(expIds[i] / 1E5) == dayObs:
            sep = np.sqrt(((deltaRas[i] * np.cos(point_decs[i] * np.pi / 180.0))**2 + deltaDecs[i]**2))
            separation.append(sep)
    if len(separation) > 20:            
        plt.hist(separation, bins = 50, range=(0,100), alpha=0.5, label=f"{dayObs}")
    dayObs = calcNextDay(dayObs)
plt.xlim(0, 100)
#plt.legend()
plt.title(f"On sky pointing model error, {startDay}-{endDay}")
plt.xlabel("Error (arseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/OnSky_Pointing_Errors_{startDay}_{endDay}.png")

In [ ]:
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}

deltaAzs = []
deltaEls = []
point_azs = []
point_els = []
true_azs = []
true_els = []
point_rots = []
good_expIds = []
for index in range(len(true_ras)):
    if (abs(deltaRas[index]) > 100.0) or abs((deltaDecs[index]) > 100.0):
        continue
    if (expIds[index] > 2025081200000) and (expIds[index] < 2025081300999):
        continue
    skyLocation = SkyCoord(true_ras[index]*u.deg, true_decs[index]*u.deg)
    time = Time(times[index], format='mjd', scale='tai')
    pressure = pressures[index] * u.pascal
    temperature = temps[index] * u.Celsius
    hum = hums[index]
    wl = wavelengths[filters[index]] * u.angstrom
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz = skyLocation.transform_to(altAz)
    true_az = Angle(obsAltAz.az.deg * u.deg)
    wrapped_true_az = true_az.wrap_at(360.0 * u.deg)
    az = Angle(azs[index] * u.deg)
    wrapped_az = az.wrap_at(360.0 * u.deg)
    deltaAz = az.deg - true_az.deg

    if deltaAz > 360.0:
        deltaAz -= 360.0
    if deltaAz < -180.0:
        deltaAz += 360.0

    deltaAz *= 3600.0
    true_el = obsAltAz.alt.deg
    deltaEl = (els[index] - true_el) * 3600.0
    #print(expIds[index], deltaAz, deltaEl)
    #print(expIds[index], wrapped_az.deg, wrapped_true_az.deg)
    if (abs(deltaAz) > 250.0) or (abs(deltaEl) > 250.0):
        continue
    true_azs.append(wrapped_true_az.deg)
    true_els.append(true_el)
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
    point_azs.append(wrapped_az.deg)
    point_els.append(els[index])
    point_rots.append(rots[index])
    good_expIds.append(expIds[index])
    
len(point_els)        

## Get camera elevation LUT.

In [ ]:
def calculateHexMotion(lut_data, el_grid):
    # The below image motion coefficients were calculated
    # with a Batoid simulation by Josh Meyers
    camHexXY = 1.00  # microns(image) / micron(hexapod)
    camHexUV = 4.92  # microns(image) / arcsecond(hexapod)
    m2HexXY = 1.13  # microns(image) / micron(hexapod)
    m2HexUV = 37.26  # microns(image) / arcsecond(hexapod)

    # Convert these to image impact in arcseconds
    # The 10.0 is microns / pixel
    # Still need to double check signs!!
    pixelScale = 0.2  # arcseconds / pixel - find this elsewhere?
    camHexXY = camHexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    camHexUV = camHexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    camCoefsX = [-camHexXY, 0, 0, 0, -camHexUV, 0]
    camCoefsY = [0, -camHexXY, 0, camHexUV, 0, 0]
    m2HexXY = m2HexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    m2HexUV = m2HexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    m2CoefsX = [-m2HexXY, 0, 0, 0, -m2HexUV, 0]
    m2CoefsY = [0, -m2HexXY, 0, m2HexUV, 0, 0]

    shiftXs = []
    shiftYs = []
    for i in range(5):
        lut_coeffs = lut_data['camera_config']['elevation_coeffs'][i]
        delta = np.polyval(lut_coeffs[::-1], el_grid)  # µm
        if i > 2:    
            delta *= 3600
        shiftXs.append(delta * camCoefsX[i])
        shiftYs.append(delta * camCoefsY[i])
    for i in range(5):
        lut_coeffs = lut_data['m2_config']['elevation_coeffs'][i]
        delta = np.polyval(lut_coeffs[::-1], el_grid)  # µm
        if i > 2:
            delta *= 3600
            print(i, delta[0], delta[-1])
        shiftXs.append(delta * m2CoefsX[i])
        shiftYs.append(delta * m2CoefsY[i])
    return [shiftXs, shiftYs]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import yaml
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)

labels = ['camX', 'camY', 'camZ', 'camRx', 'camRy', 'm2X', 'm2Y', 'm2Z', 'm2Rx', 'm2Ry']
el_grid = np.linspace(5, 85, 76)
[shiftXs, shiftYs] = calculateHexMotion(lut_data, el_grid)
for i in range(10):
    if i < 5:
        ls = '-'
    else:
        ls = '--'
    plt.plot(el_grid, shiftYs[i], label=labels[i], ls=ls)
plt.legend()
plt.xlabel("Elevation (degrees)")
plt.ylabel("Image shift (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/LUT_Az_Impact.png")

In [ ]:
amp = -23.5
off = 75.0
amp2 = 23.5
off2 = 125.0
#xs = np.linspace(0, 360, 200)
#ys = off + amp * np.sin((xs) * np.pi / 180.0)
#ys2 = off2 + amp2 * np.cos((xs) * np.pi / 180.0)

plt.figure(figsize=(8,8))
plt.suptitle(f"LSST Delta AltAz {dayObs}")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(point_els, deltaEls, c=point_azs, cmap=plt.cm.coolwarm)
#plt.plot(el_grid, dy, color='k', ls='--', label="Cam-dY\nLUT")
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta El arcsec')
plt.legend(loc='lower left')
plt.subplot(2,2,2)
p2 = plt.scatter(point_azs, deltaEls,c=point_els, cmap=plt.cm.coolwarm)
plt.plot(xs, ys, ls = '--', color='black')
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(point_els, deltaAzs, c=point_azs, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta Az arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(point_azs, deltaAzs,c=point_els, cmap=plt.cm.coolwarm)
plt.plot(xs, ys2, ls = '--', color='black')
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('Delta Az arcsec')
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Delta_AltAz_{startDay}_{endDay}.png")


In [ ]:
amp = 23.5
off = 75.0
amp2 = 23.5
off2 = 125.0
xs = np.linspace(0, 360, 200)
ys = off + amp * np.sin((xs) * np.pi / 180.0)
ys2 = off2 + amp2 * np.cos((xs) * np.pi / 180.0)

labels = ['camX', 'camY', 'camZ', 'camU', 'camV', 'm2X', 'm2Y', 'm2Z', 'm2U', 'm2V']
el_grid = np.linspace(5, 85, 76)
[shiftXs, shiftYs] = calculateHexMotion(lut_data, el_grid)
dy = -shiftYs[1] - shiftYs[3] - shiftYs[7] - shiftYs[8] + 10.0

plt.figure(figsize=(12,6))
plt.suptitle(f"LSST DeltaEl {startDay} - {endDay2}", fontsize=18)
plt.subplots_adjust(wspace=0.3)
plt.subplot(1,2,1)
p1 = plt.scatter(point_els, deltaEls, c=point_azs, cmap=plt.cm.coolwarm)
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.text(10, 180, f"{len(point_els)} data points")
plt.ylabel('Delta El (arcsec)')
plt.legend(loc='lower left')
plt.subplot(1,2,2)
mod_deltaEls = deltaEls + amp * np.sin((np.array(point_azs)) * np.pi / 180.0)
p1 = plt.scatter(point_els, mod_deltaEls, c=point_azs, cmap=plt.cm.coolwarm)
plt.plot(el_grid, dy, color='k', ls='--', label="-CamY-CamRx-M2Y-M2Rx LUT")
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.text(10, 180, f"{len(point_els)} data points")
plt.ylabel(f'Delta El modifed by AW={amp:.1f} (arcsec)')
plt.legend(loc='lower left')
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/DeltaEl_{startDay}_{endDay}.png")

In [ ]:
for i, dEl in enumerate(mod_deltaEls):
    if dEl > 150.0:
        print(good_expIds[i])

In [ ]:
separation = []
for i in range(len(point_azs)):
    sep = np.sqrt(((deltaAzs[i] * np.cos(point_els[i] * np.pi / 180.0))**2 + deltaEls[i]**2))
    separation.append(sep)
        
plt.hist(separation, bins = 50, range=(0,300), alpha=0.5)

plt.xlim(0, 300)
plt.legend()
plt.title(f"AzEl pointing model error, {dayObs}")
plt.xlabel("Error (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/AzEl_Pointing_Errors_{startDay}_{endDay}.png")

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.scatter(np.array(point_azs) * np.pi / 180.0, 90.0 - np.array(point_els), marker = 'x')
ax.set_rmax(60.0)
r_values = [10.0, 20.0, 30.0, 40.0, 50.0]
r_labels = [80.0, 70.0, 60.0, 50.0, 40.0]
ax.set_rgrids(r_values, r_labels)
ax.grid(True)
plt.savefig(f"/home/c/cslage/u/LSSTCam/data/Sky_Coverage_{startDay}_{endDay}.png")

# Generate a tpoint input file with these errors

In [ ]:
outfilename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Input_EFD_Rot_15Jul-05Aug25.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope file,Aug 19, 2025 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")

for i in range(len(point_els)):
    outfile.write(f"{true_azs[i]:.9f}\t{true_els[i]:.6f}\t{point_azs[i]:.9f}\t{point_els[i]:.9f}\t{point_rots[i]:.9f}\n")
outfile.write("END\n")
outfile.close()



In [ ]:
# The below image motion coefficients were calculated
# with a Batoid simulation by Josh Meyers
camHexXY = 1.00  # microns(image) / micron(hexapod)
camHexUV = 4.92  # microns(image) / arcsecond(hexapod)
m2HexXY = 1.13  # microns(image) / micron(hexapod)
m2HexUV = 37.26  # microns(image) / arcsecond(hexapod)

# Convert these to image impact in arcseconds
# The 10.0 is microns / pixel
# Still need to double check signs!!
pixelScale = 0.2  # arcseconds / pixel - find this elsewhere?
camHexXY = camHexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
camHexUV = camHexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
camCoefsX = [-camHexXY, 0, 0, 0, -camHexUV, 0]
camCoefsY = [0, -camHexXY, 0, camHexUV, 0, 0]
m2HexXY = m2HexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
m2HexUV = m2HexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
m2CoefsX = [-m2HexXY, 0, 0, 0, -m2HexUV, 0]
m2CoefsY = [0, -m2HexXY, 0, m2HexUV, 0, 0]
names = ['X', 'Y', 'Z', 'U', 'V']
for i in range(5):
    print(f"Cam{names[i]} coefX = {camCoefsX[i]:.3f}, coefY = {camCoefsY[i]:.3f}")
for i in range(5):
    print(f"M2{names[i]} coefX = {m2CoefsX[i]:.3f}, coefY = {m2CoefsY[i]:.3f}")

In [ ]:
len(point_ras)

In [ ]:
len(separation)

In [ ]:
dayObs = startDay
separation = []
for i in range(len(point_ras)):
    sep = np.sqrt(((deltaRas[i] * np.cos(point_decs[i] * np.pi / 180.0))**2 + deltaDecs[i]**2))
    separation.append(sep)
        
plt.hist(separation[0:233], bins = 50, range=(0,200), alpha=0.5, label='seqNum < 367')
plt.hist(separation[233:-1], bins = 50, range=(0,200), alpha=0.5, label='seqNum >= 367')

plt.xlim(0, 200)
plt.legend()
plt.title(f"On sky pointing model error, {dayObs}")
plt.xlabel("Error (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/OnSky_Pointing_Errors_Split_{dayObs}.png")

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.scatter(np.array(point_azs[0:233]) * np.pi / 180.0, 90.0 - np.array(point_els[0:233]), marker = 'x', label='seqNum < 367')
ax.scatter(np.array(point_azs[233:-1]) * np.pi / 180.0, 90.0 - np.array(point_els[233:-1]), marker = 'x', label='seqNum >= 367')
ax.set_rmax(60.0)
r_values = [10.0, 20.0, 30.0, 40.0, 50.0]
r_labels = [80.0, 70.0, 60.0, 50.0, 40.0]
ax.set_rgrids(r_values, r_labels)
ax.grid(True)
ax.legend()
plt.savefig(f"/home/c/cslage/u/LSSTCam/data/Sky_Coverage_Split_{startDay}_{endDay}.png")